In [2]:
import dspy
import json
from openai import OpenAI

with open("credentials-openai.json", "r") as fid:
    credentials = json.load(fid)

client = OpenAI(api_key=credentials["api_key"])

def send_openai_message(messages, client, model):
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=0.,
    )
    return response


In [7]:
turbo = dspy.OpenAI(model='gpt-4-0125-preview', api_key=credentials["api_key"])
dspy.settings.configure(lm=turbo)

In [12]:
class BasicQA(dspy.Signature):
    """Identify sequences through a sequential learning scheme."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="several 20-character sequences each comprised of 12 A tokens and 8 B tokens")

In [13]:
# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

dev_question = "I have an oracle that takes in a sequence of 20 characters and produces a distance from the desired result. The input sequences are comprised of permutations of 12 A characters and 8 B characters. Please propose five of these 20-character sequences (in the format ABBA...) so I can evaluate them with the oracle. Remember they must be 20 characters long and should have 12 A characters."

# Call the predictor on a particular input.
pred = generate_answer(question=dev_question)

# Print the input and the prediction.
print(f"Question: {dev_question}")
print(f"Predicted Answer: {pred.answer}")

Question: I have an oracle that takes in a sequence of 20 characters and produces a distance from the desired result. The input sequences are comprised of permutations of 12 A characters and 8 B characters. Please propose five of these 20-character sequences (in the format ABBA...) so I can evaluate them with the oracle. Remember they must be 20 characters long and should have 12 A characters.
Predicted Answer: AAAABBBBAAAABBBBAAAA
AAABBBBAAAABBBBAAAAA
AABBAABBAABBAABBAABB
BBBAAAABBBBAAAABAAAA
AABBBAAABBBAAABBBAAA


In [14]:
turbo.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: several 20-character sequences each comprised of 12 A tokens and 8 B tokens

---

Question: I have an oracle that takes in a sequence of 20 characters and produces a distance from the desired result. The input sequences are comprised of permutations of 12 A characters and 8 B characters. Please propose five of these 20-character sequences (in the format ABBA...) so I can evaluate them with the oracle. Remember they must be 20 characters long and should have 12 A characters.
Answer: AAAABBBBAAAABBBBAAAA
AAABBBBAAAABBBBAAAAA
AABBAABBAABBAABBAABB
BBBAAAABBBBAAAABAAAA
AABBBAAABBBAAABBBAAA


In [15]:
# Define the predictor. Notice we're just changing the class. The signature BasicQA is unchanged.
generate_answer_with_chain_of_thought = dspy.ChainOfThought(BasicQA)

# Call the predictor on the same input.
pred = generate_answer_with_chain_of_thought(question=dev_question)

# Print the input, the chain of thought, and the prediction.
print(f"Question: {dev_question}")
print(f"Thought: {pred.rationale.split('.', 1)[1].strip()}")
print(f"Predicted Answer: {pred.answer}")

Question: I have an oracle that takes in a sequence of 20 characters and produces a distance from the desired result. The input sequences are comprised of permutations of 12 A characters and 8 B characters. Please propose five of these 20-character sequences (in the format ABBA...) so I can evaluate them with the oracle. Remember they must be 20 characters long and should have 12 A characters.
Thought: We need to create sequences that are 20 characters long with exactly 12 A characters and 8 B characters. Here are five possible permutations:
Predicted Answer: AABBAABBAAABABABABAB, ABBABAABAAABABBABAAB, AAABABABABBABAABABAB, AABAABABABBABAABABAB, ABABABABAABAABABABAB


In [ ]:
from dspy.teleprompt import BootstrapFewShot


class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)


# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)